In [63]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import numpy
import scipy
import matplotlib as mpl
import matplotlib.dates as mpd
import pylab as plt
import datetime
import os,sys
#
import subprocess
import requests
import tarfile
import shutil
#
import urllib
import contextlib
# import urllib.request as request
# from contextlib import closing
#
import re
#
import json
import netCDF4
#

In [136]:
class NML(dict):
    #
    def __init__(self):
        self.__dict__.update({ky:vl for ky,vl in locals().items() if not ky in ('self', '__class__')})
    
    #
    def assign(self, ky1, ky2, val):
        # a "safe" assignment operator. If the keys do not exist, fail and raise an exception.
        if ky1 in self.keys() and ky2 in self[ky1].keys():
            self[ky1][ky2] = val
            #
            return True
        else:
            err_str = '{} not in self[{}].keys()'.format(ky2, ky1)
            if not(ky1) in self.keys():
                err_str = "{} not in self.keys()"
            #
            raise NameError(err_str)
            return False
    #
    def nml_to_json(self, nml_in=None, json_out=None):
        '''
        # convert nml to dict; save dict as class member; if json_out is not None,
        # export json to json_out
        #
        # NOTE: The format for .nml does not appear to be very strictly defined. Basically, it is not
        #. well defined if key-value pairs are separated by CRLF ('\n') or comma ',' -- oh, and also commas
        #. are allowed (in some cases) withink values. So this might be a work in progress, to be able
        #. to generally define k-v pairs. Right now, its success rests a little bit on, "please, all that
        #. is holy, let them not allow this or that..."
        #
        # @nml_in: filenname of input nml
        # @json_out: filename of output json file.
        #. 
        '''
        #
        if nml_in is None:
            nml_in = self.nml
        #
        #nml_dict = {}
        group_name = ''
        #k_group = 0
        #
        values_out = {}
        ky = ''
        
        with open(nml_in) as fin:
            for rw in fin:
                rw = rw.strip()
                #
                # TODO: maye we should retain comments? if so, we'll need to switch the nested
                #. structure to a list (-like), instead of a dict., to allow for multiple commented-out
                #  entries.
                #
                # for a mid-line comment:
                rw = rw.split('!')[0]
                #
                if len(rw)==0 or rw[0] in ['!', '\n']:
                    continue
                #
                if rw[0]=='&':
                    # new group:
                    if not group_name == '':
                        self[group_name] = values_out
                        #nml_dict[group_name]=values_out
                    #
                    group_name = rw[1:].strip()
                    values_out = {}
                    ky=''
                    val=''
                    #if not group_name in nml_dict.keys():
                    #    nml_dict[group_name]={}
                    continue
                    #
                
                #
                #print('** ', rw)
                #
                # TODO: this logic is almost right, and will probably work most of the time, but
                #. it might be better to be more robust about allowing multi-line entries. Note
                #. this will not work properly for a multi-entry line that end in a multi-line entry.
                if rw.startswith('/'):
                    values_out[ky]=val.strip()
                    continue
                #
                ky_vl = rw.split('=')
                if len(ky_vl) == 1:
                    #print('*** debug: ', val, ky_vl)
                    val = val + ky_vl[0]
                    continue
                elif len(ky_vl) == 2:
                    if not ky=='':
                        values_out[ky]=val.strip()
                    ky,val = [s.strip() for s in ky_vl]
                elif len(ky_vl) > 2:
                    # there are multiple entries, presumably separated by commas?? so
                    # key=val,key=val,key=val...
                    #print('** DEBUG: ', [s.strip() for s in re.split('=|,', rw) if not s.strip()==''])
                    values_out.update(dict(numpy.reshape([s.strip()
                                        for s in re.split('=|,', rw) if not s.strip()==''], (-1,2))))
                
            #
        #
        #self.nml_dict=nml_dict
        #self.update(nml_dict)
        #
        if not json_out is None:
            with open(json_out, 'w') as fout:
                #json.dump(nml_dict, fout)
                # can we just dump self?
                json.dump(self, fout)
                #json.dump({ky:vl for ky,vl in self.items()})
            #
        #
        return None
        #return nml_dict
    #
    
    #
    def json_to_nml(self, nml_out='input.nml', json_in=None, indent=None, file_mode='w'):
        '''
        # convert json or dict to an nml. export to nml_out.
        # TODO: continue to evaluate how lists, tuples, etc. are encoundered and handled. For example,
        #  we want output to be like, format = 6,8 , not format = [6,8]. so far, i don't see any "[]"
        #. characters in .nml files, so we can probably just get rid of them, but it might be smarter
        #. to just recognize when we have a list type... or to enforce that all values are saved internally
        #  as strings... The latter may become necessary, since there does not appear to be a good standard
        #. for comma, space, etc. separating values (or fields).
        '''
        #
        if json_in is None:
            json_in = self
        #
        if indent is None:
            indent = 4*chr(32)
            #
        #
        if isinstance(json_in, str):
            with open(json_in, 'r') as fin:
                json_in = json.load(fin)
            #
        #
        with open(nml_out, file_mode) as fout:
            for group,entries in json_in.items():
                fout.write('&{}\n'.format(group))
                #
                for entry,val in entries.items():
                    fout.write('{}{} = {}\n'.format(indent, entry, val ))
                fout.write('/\n\n')
                
                #
            #
            
                    
class NML_from_nml(NML):
    def __init__(self, input_nml, output=None):
        #
        nml_out=output
        super(NML_from_nml,self).__init__()
        self.__dict__.update({ky:vl for ky,vl in locals().items() if not ky in ('self', '__class__')})
        #
        # is this useful?
        #with open(input_nml) as fin:
        #    self.nml = fin.read()
        #
        self.nml_to_json(input_nml, json_out=output)
        
    #
class NML_from_json(NML):
    def __init__(self, input_json, output=None):
        #
        json_out = output
        super(NML_from_json,self).__init__()
        self.__dict__.update({ky:vl for ky,vl in locals().items() if not ky in ('self', '__class__')})
        #
        with open(input_json) as fin:
            #self.nml_dict = json.load(fin)
            self.update(json.load(fin))
        #
        if not output is None:
            self.nml = self.json_to_nml(self.nml_json, output)

            

In [93]:
cwd = os.getcwd()
root_dir = os.path.dirname(os.path.abspath(cwd))
ROOT_DIR = os.path.dirname(os.path.abspath(".."))
#
print('** \n{}\n{}\n{}'.format(cwd, root_dir, ROOT_DIR))

** 
/Users/myoder96/Codes/AM4_runtime
/Users/myoder96/Codes
/Users/myoder96


In [257]:
class AM4_batch_scripter(object):
    mpi_execs = {'mpirun':{'exec': 'mpirun', 'ntasks':'--np ', 'cpu_per_task':'-d '},
                 'srun':{'exec':'srun', 'ntasks':'--ntasks=', 'cpu_per_task':'--cpus-per-task='}}
    #
    HPC_configs={
    'mazama_hpc':{'cpu_per_node':24, 'cpu_slots':2, 'cpu_make':'intel', 'cpu_gen':'haswell',
                  'mem_per_node':64, 'modules':['intel/19', 'openmpi_3/', 'gfdl_am4/']},
    'sherlock2_hpc':{'cpu_per_node':24, 'cpu_slots':2, 'cpu_make':'intel', 'cpu_gen':'skylake',
                     'mem_per_node':192},
    'sherlock2_hpc2':{'cpu_per_node':24, 'cpu_slots':2, 'cpu_make':'intel', 'cpu_gen':'skylake',
                      'mem_per_node':384},
    'sherlock3_base':{'cpu_per_node':24, 'cpu_slots':1, 'cpu_make':'AMD', 'cpu_gen':'EPYC_7502',
                      'mem_per_node':256},
    'sherlock3_perf':{'cpu_per_node':128, 'cpu_slots':2, 'cpu_make':'AMD', 'cpu_gen':'EPYC_7742',
                      'mem_per_node':1024}
    }
    #               
    def __init__(self, batch_out='am4_batch.sh', work_dir=None, mpi_exec='mpirun',
                 input_data_path=None, input_data_tar=None, input_data_url=None,
                 nml_template='nml_input_template.nml', modules=None, diag_table_src='diag_table_v101',
                 force_copy_input=0, do_tar=0, hpc_config='mazama_hpc',
                 npes_atmos=48, nthreads_atmos=1, npes_ocean=0, job_name='AM4_run', sbatch_options_str='',
                 copy_timeout=6000):
        '''
        # parameters? input data file?
        #
        # not sure what this looks like yet, but... This script/class will be called by a wrapper
        #.  script. This process will constitute a step in a larer script (ie, each ~2 hour run in a
        #. twohour queue process).
        #. script
        # 1) review, set up, etc. the working directory
        # 2) Are the input data there?
        # 3) if not, are the input data availble?
        # 4) if not, is the tar available? if not, get it; then open, then copy.
        # 5) evaluate the input/output data. Have we achieved our objectives
        #.  (which we've not yet defined -- runtime, etc.)? Define restart as necessary 
        #.   (*** though actually, i guess this will be done by the calling script; this script will
        #.    just receive instructions).
        # 6) copy diag_table nd create input.nml
        # 7) execute MPI command
        #
        # for Mazama (note: the future will probabl hold am4.json files like, am4_mazama.json, 
        #. am4_sherlock3.json, etc.)
        #
        # current and local root path(s):
        #
        # @n_tasks: I believe n_tasks needs to be an integer multiple of 6, ie 6 faces to a cube.
        # (so any threads work on a single task/cube face). For now, let's stick with that, particularly
        #. since we'll probably usually use nthreads=1
        #. 
        '''
        #        #
        if isinstance(hpc_config, str):
            if hpc_config.endswith('.json'):
                hpc_config=json.load(hpc_config)
            else:
                hpc_config=self.HPC_configs[hpc_config]
            #
        # else hpc_config better be a dict of HW configs.
        #
        # 
        if npes_atmos%6 != 0:
            npes_atmos += npes_atmos%6
            print('*** WARNING: npes_atmos must be a multiple of 6. Increasing tasks -> {} so that npes_atmos%6=0'.format(npes_atmosn_tasks))
        #
        # same for ocean? npes_ocean%6==0 ??? right now, we're not doing ocean, so...
        # TODO: we can also check to see that we don't ask for more threads than we can get, but since
        #. these will almost always be single-threaded, let's hold off...
        #
        cwd = os.getcwd()
        root_path = os.path.dirname(os.path.abspath(cwd))
        default_data_path = os.path.join(root_path, 'AM4_data', 'AM4_run')
        #
        if input_data_path is None:
            input_data_path = default_data_path
        input_data_tar = (input_data_tar or os.path.join(os.path.dirname(input_data_path), 'AM4_run.tar.gz') )
        if work_dir is None:
            work_dir = os.path.join(cwd, 'workdir')
        #
        input_data_url = input_data_url or 'ftp://nomads.gfdl.noaa.gov/users/Ming.Zhao/AM4Documentation/GFDL-AM4.0/inputData/AM4_run.tar.gz'
        # TODO: also download check validations:
        # wget ftp://nomads.gfdl.noaa.gov/users/Ming.Zhao/AM4Documentation/GFDL-AM4.0/inputData/AM4_run.tar.gz.sha256
        # wget ftp://nomads.gfdl.noaa.gov/users/Ming.Zhao/AM4Documentation/GFDL-AM4.0/inputData/AM4_run.tar.gz.sig
        # sha256sum -c AM4_run.tar.gz.sha256
        #gpg --verify AM4_run.tar.gz.sig
        # 
        # TODO: How to handle the nml file? here as part of the batch, or in a wrapper that handles both?
        #. or some combination
        #
        if modules is None:
            modules = hpc_config.get('modules', ['intel/19', 'openmpi_3/', 'gfdl_am4/'])
            #modules = ['intel/19', 'openmpi_3/', 'gfdl_am4/']
        #
        if input_data_path is None:
            input_data_path = '{}/data/AM4_run'.format(root_path)
        input_data_tar = input_data_tar or os.path.join(root_path, 'data', 'AM4_run.tar.gz')
        #
        if isinstance(mpi_exec, dict):
            ky,vl = mpi_exec.items()[0]
            #
            # multi-category dict?
            if isinstance(vl, dict):
                self.mpi_execs.update(mpi_exec)
                mpi_exec = vl
            else:
                # single-entry dict; add this full mpi_exec entry to mpi_execs
                ky = mpi_exec['exec']
                self.mpi_execs[ky] = mpi_exec.copy()
            #
        else:
            mpi_exec = self.mpi_execs[mpi_exec]
        #
        #
        # download, untar, copy input data:
        
        #
        self.__dict__.update({ky:vl for ky,vl in locals().items() if not ky in ('self', '__class__')})
        #
        layout_1 = get_layouts(self.ntasks_total)[0]
        layout_io_1 = get_io_layouts(layout_1)[0]
        layout_2 = get_layouts(self.ntasks_total/6)[0]
        layout_io_2 = get_io_layouts(layout_2)[0]
        self.__dict__.update({ky:vl for ky,vl in locals().items() if not ky in self.__dict__.keys()})
        
        # write an input.nml file and copy other files?
        #
        #
        # write a batch file:
        # self.write_batch_script()
        
    #
    # NOTE: these nvcpu calculators will likely not work well for not-simple configurations,
    #. but again, I think most of these jobs will be single-threaded.
    @property
    def cpus_total(self):
        # NOTE: this is cpus, but not tasks. it should tell us how many cores we'll be expecting,
        #. but probably is not something submitted to SLURM unless there are constraints on nodes.
        return self.npes_atmos * self.nthreads_atmos + self.npes_ocean
    @property
    def ntasks_total(self):
        return self.npes_atmos + self.npes_ocean
    @property 
    def n_tasks(self):
        return self.ntasks_total
    @property
    def n_threads(self):
        return self.nthreads_atmos
    #
    #def layout_to_str(self, layout):
    #    return '{},{}'.format(*layout)
    #
    @property
    def default_nml_reconfig(self):
        # TODO: check these. if we're actually using ocean... is ocean supposet to be n*x=npes_ocean?
        #.  npes_total???
#         return {'coupler_nml':{'atmos_npes':self.npes_atmos, 'atmos_nthreads':self.nthreads_atmos,
#                               'ocean_npes':self.npes_ocean},
#                'fv_core_nml':{'layout':self.layout_2, 'io_layout':self.layout_io_2},
#                'ice_model_nml':{'layout':self.layout_1, 'io_layout':self.layout_io_1},
#                'land_model_nml':{'layout':self.layout_2, 'io_layout':self.layout_io_2},
#                'ocean_model_nml':{'layout':self.layout_1, 'io_layout':self.layout_io_1},
#                }
        return {'coupler_nml':{'atmos_npes':self.npes_atmos, 'atmos_nthreads':self.nthreads_atmos,
                              'ocean_npes':self.npes_ocean},
               'fv_core_nml':{'layout':'{},{}'.format(*self.layout_2),
                              'io_layout':'{},{}'.format(*self.layout_io_2)},
               'ice_model_nml':{'layout':'{},{}'.format(*self.layout_1),
                              'io_layout':'{},{}'.format(*self.layout_io_1)},
               'land_model_nml':{'layout':'{},{}'.format(*self.layout_2),
                              'io_layout':'{},{}'.format(*self.layout_io_2)},
               'ocean_model_nml':{'layout':'{},{}'.format(*self.layout_1),
                              'io_layout':'{},{}'.format(*self.layout_io_1)},
                'atmos_model_nml':{'nxblocks':1, 'nyblocks':self.nthreads_atmos}
               }
    #
    def get_input_data(self, work_dir=None, input_dir=None, input_tar=None, input_data_url=None,
                       force_copy=None, verbose=0, copy_timeout=None, diag_table_src=None):
        if work_dir is None:
            work_dir = self.work_dir
        force_copy = force_copy or self.force_copy_input
        input_dir = input_dir or self.input_data_path
        input_tar = input_tar or self.input_data_tar
        input_data_url = input_data_url or self.input_data_url
        copy_timeout = copy_timeout or self.copy_timeout
        diag_table_src = diag_table_src or self.diag_table_src
        #
        # root-dir for tar file:
        if not os.path.isdir(os.path.dirname(input_tar)):
            os.makedirs(os.path.dirname(input_tar))
        #
        # It is possible we might create work_dir in advance to add some files, but not copy all the data,
        #. so let's use INPUT as our test case.
        # TODO: better tests to (not) copy
        #input_dir = os.path.join(work_dir, 'INPUT')
        #
        print('*** DEBUG: os.path.isdir(os.path.join(work_dir, \'INPUT\'))={}'.format(os.path.isdir(os.path.join(work_dir, 'INPUT'))))
        #
       # what is a minimum file-count for INPUT? the default input config is ~500 files.
        if not os.path.isdir(work_dir):
            os.makedirs(work_dir)
        #
        if not os.path.isdir(os.path.join(work_dir, 'INPUT')) or len(os.listdir(os.path.join(work_dir, 'INPUT')))<100 or force_copy:
            # work_dir is not there or not complete, so
            # we're going to copy data from the input source to the workdir.
            # Do we have an input source?
            #
            # TODO: check for isdir(.input_path), then for .tar here.
            if not os.path.isdir(input_dir):
                # input data are not available; go get them!
                #
                if verbose:
                    print('*** VERBOSE: os.path.isfile(input_tar)={}'.format(os.path.isfile(input_tar)))
                if not os.path.isfile(input_tar):
                    # No TAR file avilble; we'll have to get it...
                    # TODO: consider downloading data first, then opening and writing file to mitigate
                    #. opportunities for contamination.
                    if verbose: print('*** VERBOSE: fetching data from {}'.format(input_data_url))
                    #
                    # NOTE: requests. does not support ftp, so use urllib or urllib2
                    with contextlib.closing(urllib.request.urlopen(input_data_url)) as url_fin:
                        with open(input_tar, 'wb') as fout:
                            shutil.copyfileobj(url_fin, fout)
                #
                if verbose:
                    print('*** VERBOSE: untarring * {} * to: {} '.format(input_tar, input_dir))
                with tarfile.open(input_tar, 'r:gz') as tar:
                    # this is another one of those cases where a shell call might be easier...
                    tar.extractall(path=os.path.dirname(input_dir))
            #
            if verbose:
                print('*** VERBOSE: copying input data from {} to {}'.format(input_dir, work_dir))
            #
            # Nuke existing workdir? for now, let's not...
            # TODO: shutil.copy2() should work for this, but it's being tempermental, maybe about the
            #. directory. So for now, let's just subprocess the shell command and revisit later. It could
            #. be a versioing problem (subprocess and related libraries are pretty version squirrelie).
            #src = input_dir
            #dst = work_dir
            # TODO: The Python copy options just don't seem to work very well (maybe there's a recursive option command or 
            #. something to tell it we're copying a directory?). Let's just use a shell command.
            #shutil.copy2(input_dir, work_dir)
            cp_command = 'cp -rf {}/ {}/'.format(input_dir, work_dir)
            try:
                # NOTE: beware the timeout.... I think the default timeout is 60 seconds or something;
                #. this can take a while, so I've been using 6000 seconds as a default.
                #sp_output = subprocess.run(cp_command.split(chr(32)), check=True,
                #           stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
                sp_output = subprocess.run(cp_command.split(chr(32)), check=True,
                           capture_output=True, timeout=copy_timeout)
            except:
                #print('*** copy error: stdout: {}, sterr: {}'.format(sp_output.stdout, sp_output=stderr) )
                print('*** COPY ERROR: {}'.format(cp_command))
                raise Exception('subprocess, copy error.')
            #
            if verbose:
                print('*** VERBOSE: copy input data complete, from {} to {}'.format(input_dir, work_dir))
            #
            # copy diag_table from a source file. We're going to construct an input.nml separately.
            # NOTE: Again, might be better to do this as a shell command, for a number of reasons.
            shutil.copy(diag_table_src, os.path.append(work_dir, 'diag_table'))
            #
        if not os.path.isdir(os.path.join(work_dir, 'RESTART')):
            os.makedirs(os.path.join(work_dir, 'RESTART'))
        #
    #
    def make_NML(self, nml_template=None, nml_configs=[], nml_out=None, json_out=None):
        '''
        # Some logic to managing and writing nml output.
        # @nml_template: An nml file or .json as a starter place.
        # @ nml_configs: 1 or more dicts (allow json filenames??) with configuration(s); they will be
        #.  integrated, in order, via dict.update(nml_config)
        #
        '''
        # add any default configs:
        # NOTE: putting default_nml_reconfig first allows the user to override.
        nml_configs = [self.default_nml_reconfig] + list(nml_configs)
        nml_template = nml_template or self.nml_template
        
        #
        if nml_template.endswith('.nml'):
            f_handler = NML_from_nml
            #NML = NML_from_nml(nml_template, output=None)
        elif nml_template.endswith('.json'):
            #NML = NML_from_json(nml_template, output=None)
            f_handler = NML_from_json
            
        else:
            # guess?
            # TODO: crack it open and look for .nml or
            #NML = NML_from_nml(nml_template, json_out=None)
            with open(nml_template, 'r') as fin:
                f_handler = NML_from_json
                for rw in fin:
                    if rw.startswith('&coupler_nml'):
                        # it is probably a .nml file.
                        f_handler = NML_from_nml
                        break
                    #
                #
        NML = f_handler(nml_template, None)
            #
        #
        for config in nml_configs:
            #
            if isinstance(config, str):
                with open(config, 'r') as fin:
                    config = json.load(config)
                #
            #
            #NML.update(config)
            for ky,d in config.items():
                NML[ky].update(d)
            #
        #
        if not nml_out is None:
            NML.json_to_nml(nml_out=nml_out)
        if not json_out is None:
            with open(json_out, 'w') as fout:
                json.dump(NML, fout)
            #
        self.NML=NML
        #
        return NML  
    #
    def write_batch_script(self, fname_out=None, chdir=None, output_out=None, output_err=None,
                           mpi_exec=None):
        '''
        # Just as it sounds; write an AM4 batch script. use various inputs (prams, json, dicts?);
        #. fetch, untar, copy input data as necessary, etc.
        # Let's start by just scripting it out (mostly), and then we'll decide what to read in as
        #. input prams, what to read in from JSON (or something), etc. Note that, for now, this will
        #  focus on a SE3 Mazama build/configuration.
        '''
        #
        fname_out = fname_out or os.path.join(self.work_dir, self.batch_out)
        chdir = chdir or self.work_dir
        output_out = output_out or 'AM4_out_%j.out'
        output_err = output_err or 'AM4_out_%j.err'
        mpi_exec = mpi_exec or self.mpi_exec
        #
        with open(fname_out, 'w') as fout:
            # This section should be pretty universal:
            fout.write('#!/bin/bash\n#\n')
            #
            fout.write('#SBATCH --ntasks={}\n'.format(self.n_tasks))
            fout.write('#SBATCVH --ncpus_per_task={}\n'.format(self.nthreads_atmos))
            if not self.job_name is None or self.job_name=='':
                fout.write('#SBATCH --job-name={}\n'.format(self.job_name))
            #
            fout.write('#SBATCH --chdir={}\n'.format(chdir) )
            fout.write('#SBATCH --output={}\n'.format(output_out))
            fout.write('#SBATCH --error={}\n'.format(output_err))
            #
            # Module swill be platform dependent. We could handle this section with JSON
            #. if we wanted to.
            fout.write('#\nmodule purge\n#\n')
            #
            for m in self.modules:
                fout.write('module load {}\n'.format(m))
            #
            fout.write('#\n#\n')
            # Universal??? My guess is we can impprove performance by increasing these values on bigger
            #. memory machines.
            # these probably need to be parameterized somehow, or included in a template... or otherwise
            #. we need to figure out what they should be. I just copied them from GFDL sample scripts.
            fout.write('export KMP_STACKSIZE=512m\n')
            fout.write('export NC_BLKSZ=1M\n')
            fout.write('export F_UFMTENDIAN=big\n')
            #
            # executable will be pretty system/build dependent
            fout.write('EXECUTABLE=${AM4_GFDL_BIN}/${AM4_GFDL_EXE}\n')
            #
            # We can probably make this part pretty universal as well...
            fout.write('#\nulimit -s unlimited\n#\n')
            fout.write('#\ncd {}\n#\n'.format(self.work_dir))
            fout.write('MPI_COMMAND={} {}{} {}{} ${{EXECUTABLE}}\n#\n'.format(mpi_exec['exec'],
                                                            mpi_exec['ntasks'], self.n_tasks,
                                                            mpi_exec['cpu_per_task'], self.n_threads))
            #
            # add an error-check:
            for ln in ['if [[ %? -ne 0 ]]; then', 'echo "ERROR: Run failed." 1>&2', 
                       '"ERROR: Output from run in {}/fms.out ... or maybe in a log file" 1>&2'.format(self.work_dir),
                       'exit 1', 'fi' ]:
                fout.write('{}\n'.format(ln))
            #
        #
            
    #
def get_layouts(n_tasks=24):
    '''
    # compute possible layouts. Include (some) error checking for valid n_tasks?
    '''
    # get all integer factor pairs:
    return numpy.array(sorted([(k,int(n_tasks/k)) for k in range(1, int(numpy.ceil(n_tasks**.5)))
                               if n_tasks%k==0],
                                key = lambda rw: numpy.sum(rw)))
#
def get_io_layouts(layout):
    '''
    # AM4 io_layouts. the second "y" term must be an integer factor of the "y" term of the input layout.
    #. Don't yet understand the first term, so for now let's limit it to 1.
    '''
    #
    return numpy.array(sorted([(1,int(layout[1]/k)) for k in range(1, int(numpy.ceil(layout[1]**.5)))
                               if layout[1]%k==0], 
                              key=lambda rw:numpy.sum(rw)))

In [258]:
print('a string with {} in {{$braces}}'.format('SOME'))

a string with SOME in {$braces}


In [259]:
#work_dir = 
ABS = AM4_batch_scripter(input_data_path='/Users/myoder96/Codes/AM4_data2/AM4_run', 
                         work_dir='/Users/myoder96/Codes/AM4_runtime/workdir2', job_name='AM4_dev')
#
print('*** ABS variables:')
for key,val in ABS.__dict__.items():
    print('{}: {}'.format(key,val))
#

zz = ABS.get_input_data(verbose=True)


my_configs = {'coupler_nml':{'days':10, 'months':0}, 'fv_core_nml':{'npx':193, 'npy':193, 'npz':50}}
my_nml = ABS.make_NML(nml_template='input_yoder_v101.nml', nml_configs=[my_configs],
                      nml_out=os.path.join(ABS.work_dir, 'input.nml') )
# NML_from_nml('input_yoder_v101.nml')

*** ABS variables:
batch_out: am4_batch.sh
work_dir: /Users/myoder96/Codes/AM4_runtime/workdir2
mpi_exec: {'exec': 'mpirun', 'ntasks': '--np ', 'cpu_per_task': '-d '}
input_data_path: /Users/myoder96/Codes/AM4_data2/AM4_run
input_data_tar: /Users/myoder96/Codes/AM4_data2/AM4_run.tar.gz
input_data_url: ftp://nomads.gfdl.noaa.gov/users/Ming.Zhao/AM4Documentation/GFDL-AM4.0/inputData/AM4_run.tar.gz
nml_template: nml_input_template.nml
modules: ['intel/19', 'openmpi_3/', 'gfdl_am4/']
diag_table_src: diag_table_v101
force_copy_input: 0
do_tar: 0
hpc_config: {'cpu_per_node': 24, 'cpu_slots': 2, 'cpu_make': 'intel', 'cpu_gen': 'haswell', 'mem_per_node': 64, 'modules': ['intel/19', 'openmpi_3/', 'gfdl_am4/']}
npes_atmos: 48
nthreads_atmos: 1
npes_ocean: 0
job_name: AM4_dev
sbatch_options_str: 
copy_timeout: 6000
cwd: /Users/myoder96/Codes/AM4_runtime
root_path: /Users/myoder96/Codes
default_data_path: /Users/myoder96/Codes/AM4_data/AM4_run
layout_1: [6 8]
layout_io_1: [1 4]
layout_2: [2 4]
lay

In [260]:
#my_nml = NML_from_nml('input_yoder_v101.nml')
print('** my_nml[fv_core_nml]:' )
print('** ', my_nml['fv_core_nml'])
#print(my_nml.keys())

print('** batch_out: ', ABS.batch_out)
#
ABS.write_batch_script()

** my_nml[fv_core_nml]:
**  {'layout': '2,4', 'io_layout': '1,4', 'npx': 193, 'npy': 193, 'ntiles': '6,', 'npz': 50, 'k_split': '1,', 'n_split': '12,', 'a2b_ord': '4,', 'adjust_dry_mass': '.true.,', 'adj_mass_vmr': '.true.,', 'print_freq': '0,', 'grid_type': '0,', 'tau': '0.', 'do_uni_zfull': '.true.', 'n_sponge': '0', 'd2_bg_k1': '0.16', 'd2_bg_k2': '0.02', 'kord_tm': '-10', 'kord_mt': '10', 'kord_tr': '10', 'hydrostatic': '.T.', 'd_ext': '0.', 'd2_bg': '0.', 'nord': '2', 'dddmp': '0.', 'd4_bg': '0.15', 'vtdm4': '0.0', 'do_vort_damp': '.F.', 'd_con': '0.', 'hord_mt': '10', 'hord_vt': '10', 'hord_tm': '10', 'hord_dp': '10', 'hord_tr': '8', 'consv_te': '0.7', 'consv_am': '.T.', 'fill': '.T.', 'z_tracer': '.T.'}
** batch_out:  am4_batch.sh


In [240]:
print(ABS.mpi_exec)

print('MPI_COMMAND={} {}{} {}{} ${{EXECUTABLE\}}'.format(ABS.mpi_exec['exec'],
                                                            ABS.mpi_exec['ntasks'], ABS.n_tasks,
                                                            ABS.mpi_exec['cpu_per_task'], ABS.n_threads))

{'exec': 'mpirun', 'ntasks': '--np ', 'cpu_per_task': '-d '}
MPI_COMMAND=mpirun --np 48 -d 1 ${EXECUTABLE\}


In [241]:
# simplified syntax to capture both stdout and stderr (to separate outputs?). NOTE: this syntax has been
#. a moving target
#sp_output = subprocess.run('ls -lh'.split(chr(32)), check=True, capture_output=True)
#
# pipe stdout and stderr to the same output
sp_output = subprocess.run('ls -lh /Users/myoder96/Codes/AM4_data2/AM4_run/'.split(chr(32)), check=True,
                           stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
#
print('** sp_output: \n', sp_output)
# for rw in sp_output.stdout.decode().split('\n'):
#     print*('** ', rw)
print('** ** stdout: \n', sp_output.stdout.decode())
for rw in sp_output.stdout.decode().split('\n'):
    print('** ', rw)

** sp_output: 
 CompletedProcess(args=['ls', '-lh', '/Users/myoder96/Codes/AM4_data2/AM4_run/'], returncode=0, stdout=b'total 352\ndrwxr-xr-x  502 myoder96  staff    16K Jul 14  2017 INPUT\ndrwxr-xr-x    2 myoder96  staff    64B Jul 17  2017 RESTART\n-rw-r--r--    1 myoder96  staff   283B Jul 14  2017 data_table\n-rw-r--r--    1 myoder96  staff    45K Jul 14  2017 diag_table\n-rw-r--r--    1 myoder96  staff    11K Jul 14  2017 field_table\n-rw-r--r--    1 myoder96  staff    55K Jul 14  2017 input.nml\n-rw-r--r--    1 myoder96  staff    55K Jul 14  2017 input.nml.unexpanded\n')
** ** stdout: 
 total 352
drwxr-xr-x  502 myoder96  staff    16K Jul 14  2017 INPUT
drwxr-xr-x    2 myoder96  staff    64B Jul 17  2017 RESTART
-rw-r--r--    1 myoder96  staff   283B Jul 14  2017 data_table
-rw-r--r--    1 myoder96  staff    45K Jul 14  2017 diag_table
-rw-r--r--    1 myoder96  staff    11K Jul 14  2017 field_table
-rw-r--r--    1 myoder96  staff    55K Jul 14  2017 input.nml
-rw-r--r--    1 myod

In [8]:
NML_test = NML_from_nml('input_yoder_v101.nml')
#

# for ky,vl in JJ.nml_dict.items():
#     print('** {}: {}'.format(ky,vl))


for ky in ['coupler_nml','coupler_nml', 'vegn_data_nml', 'simple_sulfate_nml' ]:
    print('*** {}:{}\n'.format(ky, NML_test[ky]))

*** coupler_nml:{'months': '1,', 'days': '0,', 'current_date': '1979,1,1,0,0,0,', 'calendar': "'julian'", 'dt_atmos': '1800,', 'dt_cpld': '7200,', 'use_lag_fluxes': '.true.', 'concurrent': '.false.', 'do_ocean': '.false.', 'ocean_npes': '0', 'atmos_npes': '48', 'atmos_nthreads': '1', 'use_hyper_thread': '.false.', 'ncores_per_node': '24'}

*** coupler_nml:{'months': '1,', 'days': '0,', 'current_date': '1979,1,1,0,0,0,', 'calendar': "'julian'", 'dt_atmos': '1800,', 'dt_cpld': '7200,', 'use_lag_fluxes': '.true.', 'concurrent': '.false.', 'do_ocean': '.false.', 'ocean_npes': '0', 'atmos_npes': '48', 'atmos_nthreads': '1', 'use_hyper_thread': '.false.', 'ncores_per_node': '24'}

*** vegn_data_nml:{'vegn_to_use': "'uniform'", 'K1': '10,', 'K2': '0.1,', 'fsc_liv': '0.9,', 'fsc_wood': '0.45,', 'c1(4)': '0.3', 'c2(4)': '0.3', 'Vmax': '2.0E-5, 2.0E-5, 2.0E-5, 2.0E-5, 1.50E-5,', 'm_cond': '4., 9., 9., 7., 7.,', 'alpha_phot': '0.05, 0.06, 0.06, 0.06, 0.06,', 'gamma_resp': '0.03, 0.02, 0.02, 0.02,

In [ ]:
print(NML_test['aerosolrad_package_nml']['sulfate_indices'])

### Working NML example.

- Start with a standard template
- Compute layouts for an MPI configuration
- Modify layout variables (in internal JSON/dict)
- Export working input.nml

In [134]:
        
n_tasks = 48
n_threads = 1
#
layouts_1 = get_layouts(n_tasks)
layouts_2 = get_layouts(n_tasks = n_tasks/6)
#
layout_io_1 = get_io_layouts(layouts_1[0])
layout_io_2 = get_io_layouts(layouts_2[0])
#
print('** Layouts_1: ', layouts_1)
print('** Layouts_2: ', layouts_2)
#
print('** Layouts_io_1: ', layout_io_1)
print('** Layouts_io_2: ', layout_io_2)
#
my_nml = NML_from_nml(input_nml='input_yoder_v101.nml')

** Layouts_1:  [[ 6  8]
 [ 4 12]
 [ 3 16]
 [ 2 24]
 [ 1 48]]
** Layouts_2:  [[2 4]
 [1 8]]
** Layouts_io_1:  [[1 4]
 [1 8]]
** Layouts_io_2:  [[1 4]]


In [135]:
print('*** ', my_nml.keys())

***  dict_keys([])


In [133]:
layout_1=layouts_1[0]
layout_2=layouts_2[0]
#
layout_io = layout_io_1[0]

#
# print out layouts, as they are imported:
for grp in ('fv_core_nml', 'land_model_nml','ocean_model_nml', 'ice_model_nml'):
    print('** {}::layout: {}'.format(grp, my_nml[grp]['layout']))
    print('** {}::io_layout: {}'.format(grp, my_nml[grp]['layout']))
    print('\n')

** Layouts_1:  [[ 6  8]
 [ 4 12]
 [ 3 16]
 [ 2 24]
 [ 1 48]]
** Layouts_2:  [[2 4]
 [1 8]]
** Layouts_io_1:  [[1 4]
 [1 8]]
** Layouts_io_2:  [[1 4]]


KeyError: 'fv_core_nml'

In [ ]:
print('** ', 48%6)
print('** ', 48%5)

In [ ]:
#print('** ', my_nml['fv_core_nml']['layout'])
#
for grp in ('fv_core_nml', 'land_model_nml'):
    my_nml.assign(grp, 'layout', ','.join([str(x) for x in layout_2]))
    my_nml.assign(grp, 'io_layout', ','.join([str(x) for x in layout_io]))
    print('** {}:: {}, {}'.format(grp, my_nml[grp]['layout'], my_nml[grp]['io_layout']))
#
for grp in ('ocean_model_nml', 'ice_model_nml'):
    my_nml.assign(grp, 'layout', ','.join([str(x) for x in layout_1]))
    my_nml.assign(grp, 'io_layout', ','.join([str(x) for x in layout_io]))
    print('** {}:: {}, {}'.format(grp, my_nml[grp]['layout'], my_nml[grp]['io_layout']))
#
for ky,vl in [('npx',193), ('npy', 193), ('npz', 50)]:
    my_nml.assign('fv_core_nml', ky, vl)
#
for ky,vl in [('co2_ceiling', 4800.0E-06), ('time_varying_co2', '.true.'), ('co2_base_value',348.0E-06),
            ('co2_floor', 100.0E-06), ('c02_data_source', 'namelist') ]:
    # NOTE: we want to allow new assignment:
    my_nml['radiative_gases_nml'][ky] = vl
print('*** radiative_gases: ', my_nml['radiative_gases_nml'])

In [ ]:
for ky,vl in my_nml['radiative_gases_nml'].items():
    print('** {}:: {}'.format(ky, vl))
print('\n\n')
#
for ky,vl in my_nml['fv_core_nml'].items():
    print('** {}:: {}'.format(ky, vl))

In [ ]:
my_nml.json_to_nml(nml_out='my_output.nml', json_in=my_nml)

In [ ]:
print(','.join([str(x) for x in [1,2,3]]))